# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. 
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [7]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [8]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Join regional indicators together.

#### Hamlets

In [28]:
travel = gpd.read_file("C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/outputs/ag_adm1/ag_to_HDurban_pre-F10-20-50_adm1.shp") 
travel = pd.DataFrame(travel)
isolation = gpd.read_file("C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/outputs/ag_adm1/ag_isolation_pre_adm1.shp") 
isolation = pd.DataFrame(isolation)
poverty = os.path.join(pth, "poverty and sociodem/regional_masterset.csv")
poverty = pd.read_csv(poverty)

print(travel.info(), end="\n\n")
print(isolation.info(), end="\n\n")
print(poverty.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ADM1_PCODE  14 non-null     object  
 1   ADM1_FR     14 non-null     object  
 2   pre_p       14 non-null     float64 
 3   F10_p       14 non-null     float64 
 4   F20_p       13 non-null     float64 
 5   F50_p       13 non-null     float64 
 6   dif_10b_p   14 non-null     float64 
 7   dif_20b_p   13 non-null     float64 
 8   dif_50b_p   13 non-null     float64 
 9   pc_10b_p    14 non-null     float64 
 10  pc_20b_p    13 non-null     float64 
 11  pc_50b_p    13 non-null     float64 
 12  geometry    14 non-null     geometry
dtypes: float64(10), geometry(1), object(2)
memory usage: 1.5+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ADM

In [29]:
# Make sure the ID is unique and fully matched
list1 = list(travel['ADM1_PCODE'].unique())
list2 = list(isolation['ADM1_PCODE'].unique())
list3 = list(poverty['ADM1_PCODE'].unique())
print(len(list1))
print(len(list2))
print(len(list3))

14
14
14


In [30]:
all_indicators = pd.merge(travel, isolation, on='ADM1_PCODE',how='left')
del travel, isolation # Avoiding memory issues.

In [31]:
all_indicators = pd.merge(all_indicators, poverty, on='ADM1_PCODE',how='left')
del poverty # Avoiding memory issues.
print(all_indicators.info(), end="\n\n")
print(all_indicators)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 52 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ADM1_PCODE  14 non-null     object  
 1   ADM1_FR_x   14 non-null     object  
 2   pre_p       14 non-null     float64 
 3   F10_p       14 non-null     float64 
 4   F20_p       13 non-null     float64 
 5   F50_p       13 non-null     float64 
 6   dif_10b_p   14 non-null     float64 
 7   dif_20b_p   13 non-null     float64 
 8   dif_50b_p   13 non-null     float64 
 9   pc_10b_p    14 non-null     float64 
 10  pc_20b_p    13 non-null     float64 
 11  pc_50b_p    13 non-null     float64 
 12  geometry_x  14 non-null     geometry
 13  ADM1_FR_y   14 non-null     object  
 14  orig_ct     14 non-null     float64 
 15  val_a1      14 non-null     float64 
 16  preiso      14 non-null     float64 
 17  preisoval   14 non-null     float64 
 18  F10iso      14 non-null     float64 
 19  F10isoval 

In [32]:
display(all_indicators.describe())

,pre_p,F10_p,F20_p,F50_p,dif_10b_p,dif_20b_p,dif_50b_p,pc_10b_p,pc_20b_p,pc_50b_p,...,pre_pcval,F10_pcval,F20_pcval,F50_pcval,neo_d,perinat_r,still_d,MPI_r,MPI_pc,agjob_pc
count,14.000000,14.000000,13.000000,13.000000,14.000000,13.000000,13.000000,14.000000,13.000000,13.000000,...,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,95.603386,111.008567,121.688613,111.643708,23.301022,45.229020,39.009761,30.751831,75.450145,59.131854,...,7.817976,29.906578,40.195048,53.276305,16.714286,40.714286,16.142857,0.303143,57.533571,37.242857
std,52.313168,52.741587,48.298793,58.829489,22.509590,33.812396,37.729349,26.802992,62.611825,58.620576,...,16.247840,34.709652,35.963218,35.604067,11.179848,7.680430,9.741720,0.110411,17.767042,23.241863
min,42.174238,44.572684,46.396054,46.855889,0.234355,0.344069,0.727060,0.520698,0.767220,1.642451,...,0.000000,0.641299,0.733960,5.247775,2.000000,25.000000,1.000000,0.084000,18.260000,0.700000
25%,59.946920,82.172721,91.771337,58.486423,2.666615,26.198366,9.561513,5.367714,32.344539,19.452567,...,0.133184,5.277662,8.266360,22.783712,9.500000,37.500000,11.000000,0.275500,52.532500,17.000000
50%,85.264390,109.000186,114.495134,97.608011,24.457682,38.748426,21.614515,27.719971,53.970293,36.020442,...,0.748248,13.304441,31.298815,51.633312,12.000000,40.000000,14.500000,0.301500,59.695000,37.000000
75%,107.745946,115.387377,144.858777,155.805759,28.673270,67.970736,73.887132,47.484180,100.781332,66.530442,...,2.085470,50.153302,67.210011,85.237913,20.000000,43.750000,20.500000,0.362250,69.782500,56.650000
max,208.556991,224.578642,216.596605,219.724300,76.917360,107.380360,106.329245,83.295930,195.908509,179.010508,...,46.342424,99.994627,100.000000,100.000000,46.000000,54.000000,39.000000,0.502000,85.670000,76.400000


In [33]:
all_indicators = all_indicators.drop(columns=['geometry_y', 'ADM1_FR_x', 'ADM1_FR_y']) 
all_indicators.rename(columns={'geometry_x': 'geometry'}, inplace=True) 
all_indicators.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 49 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ADM1_PCODE  14 non-null     object  
 1   pre_p       14 non-null     float64 
 2   F10_p       14 non-null     float64 
 3   F20_p       13 non-null     float64 
 4   F50_p       13 non-null     float64 
 5   dif_10b_p   14 non-null     float64 
 6   dif_20b_p   13 non-null     float64 
 7   dif_50b_p   13 non-null     float64 
 8   pc_10b_p    14 non-null     float64 
 9   pc_20b_p    13 non-null     float64 
 10  pc_50b_p    13 non-null     float64 
 11  geometry    14 non-null     geometry
 12  orig_ct     14 non-null     float64 
 13  val_a1      14 non-null     float64 
 14  preiso      14 non-null     float64 
 15  preisoval   14 non-null     float64 
 16  F10iso      14 non-null     float64 
 17  F10isoval   14 non-null     float64 
 18  F20iso      14 non-null     float64 
 19  F20isoval 

In [34]:
crs = "EPSG:4326"
all_indicators = GeoDataFrame(all_indicators, crs=crs, geometry='geometry') 
all_indicators.to_file(driver='ESRI Shapefile', filename='C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/poverty and sociodem/ag_travel_iso_pov_health.shp') 

In [35]:
# To prevent memory errors if moving straight on to next set, delete hamlet files from memory
del all_indicators